<a href="https://colab.research.google.com/github/poojanpatel2712/VLSI1/blob/main/digital-mux4x1-openlane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digital inverter with OpenLane

```
Copyright 2022 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

Run a simple digital inverter design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the [open source SKY130 PDK](https://github.com/google/skywater-pdk/).

In [6]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''

import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!python -m pip install gdstk gdstk
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


Empty environment created at prefix: /content/conda-env

Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                                                Version  Build                 Channel        Size
─────────────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
─────────────────────────────────────────────────────────────────────────────────────────────────────────────

  + open_pdks.sky130a                         1.0.456_0_g2a38e86  20231104_052339       litex-hub       1GB
  + _libgcc_mutex                                            0.1  main                  main            3kB
  + libstdcxx-ng                                          11.2.0  h1234567_1            main            6MB
  + ld_impl_linux-64                                        2.38  h1181459_1            main          749kB
  + libgcc-devel_linux-64      

## Write verilog

In [39]:
%%writefile mux4x1.v
module mux4x1 (
    input wire S1,
    input wire S0,
    input wire D0,
    input wire D1,
    input wire D2,
    input wire D3,
    output wire Y
);

assign Y = (S1 & ~S0) ? D0 :
          (S1 & S0) ? D1 :
          (~S1 & S0) ? D2 :
          D3;

endmodule

Writing mux4x1.v


## Write configuration

[Documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html)

In [40]:
%%writefile config.json
{
    "DESIGN_NAME": "mux4x1",
    "VERILOG_FILES": "dir::mux4x1.v",
    "RUN_CTS": false,
    "CLOCK_PORT": null,
    "PL_RANDOM_GLB_PLACEMENT": true,
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 40 60",
    "PL_TARGET_DENSITY": 0.75,
    "FP_PDN_AUTO_ADJUST": false,
    "FP_PDN_VPITCH": 25,
    "FP_PDN_HPITCH": 25,
    "FP_PDN_VOFFSET": 5,
    "FP_PDN_HOFFSET": 5,
    "DIODE_INSERTION_STRATEGY": 3
}

Overwriting config.json


## Run OpenLane Flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

In [41]:
%env PDK=sky130A
!flow.tcl -design .

env: PDK=sky130A
OpenLane 2023.11.03_0_gf4f8dad8-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.json'...
[INFO]: PDK Root: /content/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[WARNING]: DIODE_INSERTION_STRATEGY is now deprecated; use GRT_REPAIR_ANTENNAS, DIODE_ON_PORTS and RUN_HEURISTIC_DIODE_INSERTION instead.
[INFO]: DIODE_INSERTION_STRATEGY set to 3. Setting GRT_REPAIR_ANTENNAS to 1
[INFO]: Run Directory: /content/runs/RUN_2023.11.08_20.13.37
[INFO]: Saving runtime environment...
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[WARNING]: PNR_SDC_FILE is not set. It is recommended to write a custom SDC file 

In [16]:
!pip install libparse

## Display layout

In [42]:
import pathlib
import gdstk
import IPython.display

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('mux4x1.svg')
IPython.display.SVG('mux4x1.svg')

## Metrics

[Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)


In [43]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(reports[-1])
df.transpose()

0
design                                        /content
design_name                                     mux4x1
config                         RUN_2023.11.08_20.13.37
flow_status                             flow completed
total_runtime                               0h0m55s0ms
routed_runtime                              0h0m34s0ms
(Cell/mm^2)/Core_Util                           7500.0
DIEAREA_mm^2                                    0.0024
CellPer_mm^2                                    3750.0
OpenDP_Util                                         -1
Final_Util                                          -1
Peak_Memory_Usage_MB                             465.8
synth_cell_count                                     2
tritonRoute_violations                               0
Short_violations                                     0
MetSpc_violations                                    0
OffGrid_violations                                   0
MinHole_violations                                   0
Other_violations                                     0
Magic_violations                                     0
pin_antenna_violations                               0
net_antenna_violations                               0
lvs_total_errors                                     0
cvc_total_errors                                    -1
klayout_violations                                  -1
wire_length                                        193
vias                                                53
wns                                                0.0
pl_wns                                              -1
optimized_wns                                       -1
fastroute_wns                                       -1
spef_wns                                            -1
tns                                                0.0
pl_tns                                              -1
optimized_tns                                       -1
fastroute_tns                                       -1
spef_tns                                            -1
HPWL                                                -1
routing_layer1_pct                                 0.0
routing_layer2_pct                                3.63
routing_layer3_pct                                3.69
routing_layer4_pct                                 0.0
routing_layer5_pct                                 0.0
routing_layer6_pct                                 0.0
wires_count                                         12
wire_bits                                           12
public_wires_count                                   7
public_wire_bits                                     7
memories_count                                       0
memory_bits                                          0
processes_count                                      0
cells_pre_abc                                        6
AND                                                  1
DFF                                                  0
NAND                                                 0
NOR                                                  0
OR                                                   0
XOR                                                  0
XNOR                                                 0
MUX                                                  3
inputs                                               6
outputs                                              1
level                                                2
DecapCells                                          94
WelltapCells                                        16
DiodeCells                                           0
FillCells                                           21
NonPhysCells                                         9
TotalCells                                         140
CoreArea_um^2                                1086.0416
power_slowest_internal_uW                           -1
power_slowest_switching_uW                          -1
power_slowest_leakage_uW              